# Text-to-SQL Agentic AI System

## CrewAI-based Natural Language to SQL Agent with Banking Database

This notebook demonstrates a complete Text-to-SQL agent system that:
- Converts natural language queries to SQL
- Validates queries for safety and security
- Executes queries in a sandboxed environment
- Evaluates results against expected outputs

### Architecture

```
User Query (NL)
    ✓ LLM Interface (OpenAI GPT-4o-mini)
    ✓ SQL Generator
    ✓ SQL Validator (Security)
    ✓ Database Adapter (Safe Execution)
    ✓ Natural Language Response Generator
```

## 1. Setup and Configuration

First, let's set up the environment and load the necessary components.

In [1]:
# Import required libraries
import os
import sys
import logging
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables from .env file
load_dotenv()

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

print("✓ Environment configured")
print(f"✓ Current directory: {Path.cwd()}")
print(f"✓ OPENAI_API_KEY set: {'OPENAI_API_KEY' in os.environ}")

✓ Environment configured
✓ Current directory: c:\Users\krish\Projects\CodingAgents_AB\text_to_sql_agent\notebooks
✓ OPENAI_API_KEY set: True


In [2]:
# Import agent components
from agent import (
    DatabaseAdapter,
    SchemaLoader,
    LLMInterface,
    SQLValidator,
    TextToSQLExecutor,
    EvaluationHarness
)

print("✓ Agent components imported successfully")

✓ Agent components imported successfully


## 2. Database Schema Introspection

Let's examine the banking database schema and understand the structure.

In [4]:
# Initialize database adapter
# Use absolute path to ensure it works from notebook directory
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent  # Go up 2 levels: notebooks -> text_to_sql_agent -> project_root
db_path = project_root / 'problem_statement_task' / 'banking_insights.db'

# Alternative: if the above doesn't work, try direct absolute path
if not db_path.exists():
    db_path = Path(r'C:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\banking_insights.db')

print(f"Database path: {db_path}")
print(f"Database exists: {db_path.exists()}")
print(f"Absolute path: {db_path.resolve()}")

# Connect to database
db_adapter = DatabaseAdapter(str(db_path), readonly=True)
db_adapter.connect()

print("✓ Connected to database")

2026-02-11 17:07:09,870 - agent.db_adapter - INFO - Connected to database: c:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\banking_insights.db


Database path: c:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\banking_insights.db
Database exists: True
Absolute path: C:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\banking_insights.db
✓ Connected to database


In [5]:
# Load and display schema
schema = db_adapter.get_schema()
schema_loader = SchemaLoader(schema)

print("\n" + "="*70)
print("DATABASE SCHEMA")
print("="*70)

for table_name in sorted(schema.keys()):
    columns = schema[table_name]['columns']
    print(f"\n{table_name}:")
    for col in columns:
        pk_indicator = "[PK]" if col['pk'] else ""
        notnull_indicator = "NOT NULL" if col['notnull'] else "NULL"
        print(f"  • {col['name']:<20} {col['type']:<10} {notnull_indicator:<10} {pk_indicator}")

2026-02-11 17:07:14,644 - agent.db_adapter - INFO - Schema retrieved: 6 tables



DATABASE SCHEMA

Account:
  • AccountID            INTEGER    NULL       [PK]
  • CustomerID           INTEGER    NOT NULL   
  • BranchID             INTEGER    NOT NULL   
  • AccountType          TEXT       NOT NULL   
  • OpenDate             DATE       NOT NULL   
  • Status               TEXT       NOT NULL   

Branch:
  • BranchID             INTEGER    NULL       [PK]
  • Name                 TEXT       NOT NULL   
  • City                 TEXT       NULL       
  • State                TEXT       NULL       

Customer:
  • CustomerID           INTEGER    NULL       [PK]
  • Name                 TEXT       NOT NULL   
  • City                 TEXT       NULL       
  • State                TEXT       NULL       

Dispute:
  • DisputeID            TEXT       NULL       [PK]
  • TxnID                TEXT       NOT NULL   
  • RaisedDate           DATE       NOT NULL   
  • ReasonCode           TEXT       NOT NULL   
  • Status               TEXT       NOT NULL   
  • Resolution 

In [6]:
# Display sample data from each table
print("\n" + "="*70)
print("SAMPLE DATA")
print("="*70)

for table_name in sorted(schema.keys()):
    samples = db_adapter.get_table_sample(table_name, limit=3)
    print(f"\n{table_name} (first 3 rows):")
    if samples:
        df = pd.DataFrame(samples)
        display(df)
    else:
        print("  No data found")

2026-02-11 17:07:19,132 - agent.db_adapter - INFO - Query executed successfully, returned 3 rows



SAMPLE DATA

Account (first 3 rows):


AccountID  CustomerID  BranchID AccountType    OpenDate  Status
0       1001           1         2    Checking  2022-07-03  Closed
1       1002           2         1    Checking  2021-01-16  Active
2       1003           2         3     Savings  2023-02-19  Active

2026-02-11 17:07:19,411 - agent.db_adapter - INFO - Query executed successfully, returned 3 rows



Branch (first 3 rows):


...


Merchant (first 3 rows):


MerchantID             Name
0           1  Reliance Retail
1           2     Amazon India
2           3         Flipkart

2026-02-11 17:07:19,441 - agent.db_adapter - INFO - Query executed successfully, returned 3 rows



Transactions (first 3 rows):


TxnID  AccountID              TxnDate  AmountINR TxnType Channel  \
0  T950N0UP4D       1002  2025-03-12 12:37:00    53762.0  Credit  Online   
1  TIZRW7JAKY       1002  2025-05-19 00:19:00     2460.0   Debit  Online   
2  TRIOJW21HH       1002  2024-12-10 16:10:00    15481.0   Debit     ATM   

   MerchantID         Category      Description  
0           9           Salary           Salary  
1          11         Transfer         Transfer  
2          12  Cash Withdrawal  Cash Withdrawal

## 3. Initialize LLM and Executor

Set up the LLM interface and the main executor pipeline.

In [7]:
# Initialize LLM interface
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it before running this cell.")

llm_interface = LLMInterface(api_key=api_key, model="gpt-4o-mini", temperature=0.0)
print("✓ LLM Interface initialized (gpt-4o-mini)")

# Initialize SQL validator
sql_validator = SQLValidator(schema_allowlist=schema_loader.get_allowlist())
print("✓ SQL Validator initialized")

# Initialize executor
executor = TextToSQLExecutor(db_adapter, schema_loader, llm_interface, sql_validator)
print("✓ Text-to-SQL Executor initialized")

2026-02-11 17:07:25,126 - agent.llm_interface - INFO - LLM Interface initialized with model: gpt-4o-mini


✓ LLM Interface initialized (gpt-4o-mini)
✓ SQL Validator initialized
✓ Text-to-SQL Executor initialized


## 4. Test Single Query

Let's test the system with a sample query.

In [8]:
# Test query
test_query = "Top 5 merchants by debit spend in July 2025"

print(f"\nQuery: {test_query}\n")
print("-" * 70)

# Execute the query
result = executor.execute(test_query, include_response=True)

print(f"SQL Valid: {result['is_valid']}")
print(f"\nGenerated SQL:\n{result['sql_query']}")

if result['error']:
    print(f"\nError: {result['error']}")
else:
    print(f"\nResults ({len(result['results'])} rows):")
    if result['results']:
        df = pd.DataFrame(result['results'])
        display(df)
    
    if result['response']:
        print(f"\nNatural Language Response:\n{result['response']}")


Query: Top 5 merchants by debit spend in July 2025

----------------------------------------------------------------------


2026-02-11 17:07:30,219 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:30,229 - agent.llm_interface - INFO - Generated SQL: SELECT m.Name, SUM(t.AmountINR) AS TotalDebitSpend
FROM Transactions t
JOIN Merchant m ON t.Merchant...
2026-02-11 17:07:30,230 - agent.executor - INFO - Generated SQL: SELECT m.Name, SUM(t.AmountINR) AS TotalDebitSpend
FROM Transactions t
JOIN Merchant m ON t.Merchant...
2026-02-11 17:07:30,231 - agent.sql_validator - INFO - Schema validation passed for tables: {'Merchant', 'Transactions'}
2026-02-11 17:07:30,232 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:07:30,238 - agent.db_adapter - INFO - Query executed successfully, returned 5 rows
2026-02-11 17:07:30,240 - agent.executor - INFO - Query executed, returned 5 rows
2026-02-11 17:07:31,755 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:31,761 - agent.

SQL Valid: True

Generated SQL:
SELECT m.Name, SUM(t.AmountINR) AS TotalDebitSpend
FROM Transactions t
JOIN Merchant m ON t.MerchantID = m.MerchantID
WHERE t.TxnType = 'Debit' AND t.TxnDate >= '2025-07-01' AND t.TxnDate < '2025-08-01'
GROUP BY m.Name
ORDER BY TotalDebitSpend DESC
LIMIT 5;

Results (5 rows):


Name  TotalDebitSpend
0         IRCTC        1002190.0
1        Zomato         965256.0
2        Swiggy         965091.0
3   Apple Store         688285.0
4  Amazon India         595250.0


Natural Language Response:
In July 2025, the top 5 merchants by debit spend were:

1. IRCTC: ₹1,002,190
2. Zomato: ₹965,256
3. Swiggy: ₹965,091
4. Apple Store: ₹688,285
5. Amazon India: ₹595,250


## 5. Run Multiple Example Queries

Execute several test queries to demonstrate the system.

In [9]:
# Example queries from the test set
example_queries = [
    "Top 5 customers by total debit spend",
    "Count active accounts by account type",
    "List the top 3 spending categories by total transaction value",
    "Which branch has the highest number of customers",
    "Top 5 merchants by number of disputes (all time)"
]

print("\n" + "="*70)
print("EXECUTING EXAMPLE QUERIES")
print("="*70)

example_results = []
for i, query in enumerate(example_queries, 1):
    print(f"\n[Query {i}/{len(example_queries)}] {query}")
    print("-" * 70)
    
    result = executor.execute(query, include_response=True)
    example_results.append(result)
    
    if result['error']:
        print(f"❌ Error: {result['error']}")
    elif not result['is_valid']:
        print(f"❌ SQL Validation Failed")
    else:
        print(f"✓ Success ({len(result['results'])} rows returned)")
        print(f"\nSQL: {result['sql_query'][:60]}...")


EXECUTING EXAMPLE QUERIES

[Query 1/5] Top 5 customers by total debit spend
----------------------------------------------------------------------


2026-02-11 17:07:44,751 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:44,753 - agent.llm_interface - INFO - Generated SQL: SELECT Customer.CustomerID, Customer.Name, SUM(Transactions.AmountINR) AS TotalDebitSpend
FROM Trans...
2026-02-11 17:07:44,753 - agent.executor - INFO - Generated SQL: SELECT Customer.CustomerID, Customer.Name, SUM(Transactions.AmountINR) AS TotalDebitSpend
FROM Trans...
2026-02-11 17:07:44,754 - agent.sql_validator - INFO - Schema validation passed for tables: {'Account', 'Customer', 'Transactions'}
2026-02-11 17:07:44,755 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:07:44,758 - agent.db_adapter - INFO - Query executed successfully, returned 5 rows
2026-02-11 17:07:44,760 - agent.executor - INFO - Query executed, returned 5 rows
2026-02-11 17:07:46,527 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:46,5

✓ Success (5 rows returned)

SQL: SELECT Customer.CustomerID, Customer.Name, SUM(Transactions....

[Query 2/5] Count active accounts by account type
----------------------------------------------------------------------


2026-02-11 17:07:47,565 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:47,574 - agent.llm_interface - INFO - Generated SQL: SELECT AccountType, COUNT(*) AS ActiveAccountCount 
FROM Account 
WHERE Status = 'Active' 
GROUP BY ...
2026-02-11 17:07:47,577 - agent.executor - INFO - Generated SQL: SELECT AccountType, COUNT(*) AS ActiveAccountCount 
FROM Account 
WHERE Status = 'Active' 
GROUP BY ...
2026-02-11 17:07:47,578 - agent.sql_validator - INFO - Schema validation passed for tables: {'Account'}
2026-02-11 17:07:47,580 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:07:47,582 - agent.db_adapter - INFO - Query executed successfully, returned 2 rows
2026-02-11 17:07:47,583 - agent.executor - INFO - Query executed, returned 2 rows
2026-02-11 17:07:48,649 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:48,660 - agent.llm_interface - I

✓ Success (2 rows returned)

SQL: SELECT AccountType, COUNT(*) AS ActiveAccountCount 
FROM Acc...

[Query 3/5] List the top 3 spending categories by total transaction value
----------------------------------------------------------------------


...

✓ Success (3 rows returned)

SQL: SELECT Category, SUM(AmountINR) AS TotalSpent
FROM Transacti...

[Query 4/5] Which branch has the highest number of customers
----------------------------------------------------------------------


2026-02-11 17:07:52,102 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:52,104 - agent.llm_interface - INFO - Generated SQL: SELECT Branch.Name, COUNT(Customer.CustomerID) AS CustomerCount
FROM Branch
JOIN Account ON Branch.B...
2026-02-11 17:07:52,105 - agent.executor - INFO - Generated SQL: SELECT Branch.Name, COUNT(Customer.CustomerID) AS CustomerCount
FROM Branch
JOIN Account ON Branch.B...
2026-02-11 17:07:52,107 - agent.sql_validator - INFO - Schema validation passed for tables: {'Account', 'Branch', 'Customer'}
2026-02-11 17:07:52,109 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:07:52,111 - agent.db_adapter - INFO - Query executed successfully, returned 1 rows
2026-02-11 17:07:52,112 - agent.executor - INFO - Query executed, returned 1 rows
2026-02-11 17:07:52,999 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:53,006 - a

✓ Success (1 rows returned)

SQL: SELECT Branch.Name, COUNT(Customer.CustomerID) AS CustomerCo...

[Query 5/5] Top 5 merchants by number of disputes (all time)
----------------------------------------------------------------------


2026-02-11 17:07:54,717 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:54,724 - agent.llm_interface - INFO - Generated SQL: SELECT m.Name, COUNT(d.DisputeID) AS DisputeCount
FROM Merchant m
JOIN Transactions t ON m.MerchantI...
2026-02-11 17:07:54,727 - agent.executor - INFO - Generated SQL: SELECT m.Name, COUNT(d.DisputeID) AS DisputeCount
FROM Merchant m
JOIN Transactions t ON m.MerchantI...
2026-02-11 17:07:54,730 - agent.sql_validator - INFO - Schema validation passed for tables: {'Merchant', 'Dispute', 'Transactions'}
2026-02-11 17:07:54,732 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:07:54,738 - agent.db_adapter - INFO - Query executed successfully, returned 5 rows
2026-02-11 17:07:54,740 - agent.executor - INFO - Query executed, returned 5 rows
2026-02-11 17:07:56,330 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:07:56,3

✓ Success (5 rows returned)

SQL: SELECT m.Name, COUNT(d.DisputeID) AS DisputeCount
FROM Merch...


## 6. Full Evaluation Against Test Set

Run the complete evaluation harness against all test queries.

In [10]:
# Load test queries and sample responses
notebook_dir = Path.cwd()
project_root = notebook_dir.parent.parent  # Go up 2 levels

test_queries_path = project_root / 'problem_statement_task' / 'test_queries.csv'
sample_responses_path = project_root / 'problem_statement_task' / 'sample_queries_with_responses.csv'

# Fallback to absolute paths if needed
if not test_queries_path.exists():
    test_queries_path = Path(r'C:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\test_queries.csv')
if not sample_responses_path.exists():
    sample_responses_path = Path(r'C:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\sample_queries_with_responses.csv')

print(f"Test queries path: {test_queries_path}")
print(f"Exists: {test_queries_path.exists()}\n")

# Initialize evaluation harness
eval_harness = EvaluationHarness(executor, match_threshold=0.85)
print("✓ Evaluation Harness initialized")

# Load queries and responses
test_queries = eval_harness.load_test_queries(str(test_queries_path))
sample_responses = None
if sample_responses_path.exists():
    sample_responses = eval_harness.load_sample_responses(str(sample_responses_path))

2026-02-11 17:08:04,451 - agent.eval - INFO - Loaded 11 test queries from c:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\test_queries.csv
2026-02-11 17:08:04,454 - agent.eval - INFO - Loaded 2 sample responses from c:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\sample_queries_with_responses.csv


Test queries path: c:\Users\krish\Projects\CodingAgents_AB\problem_statement_task\test_queries.csv
Exists: True

✓ Evaluation Harness initialized


In [11]:
# Run full evaluation
print(f"\nRunning evaluation on {len(test_queries)} queries...\n")

results, summary = eval_harness.evaluate_batch(test_queries, sample_responses)

# Display summary
eval_harness.print_summary(summary)

# Display detailed results
print("\n" + "="*70)
print("DETAILED EVALUATION RESULTS")
print("="*70)

results_df = pd.DataFrame([
    {
        "Query": result['query'][:40] + "..." if len(result['query']) > 40 else result['query'],
        "SQL Valid": "✓" if result['sql_valid'] else "✗",
        "Response Match": f"{result['match_score']:.0%}" if result.get('match_score') is not None else "N/A",
        "Passed": "✓" if result['passed'] else "✗",
        "Error": result['error'][:30] + "..." if result['error'] and len(result['error']) > 30 else result['error']
    }
    for result in results
])

display(results_df)


Running evaluation on 11 queries...



2026-02-11 17:08:11,370 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:08:11,375 - agent.llm_interface - INFO - Generated SQL: SELECT m.Name, SUM(t.AmountINR) AS TotalDebitSpend
FROM Transactions t
JOIN Merchant m ON t.Merchant...
2026-02-11 17:08:11,376 - agent.executor - INFO - Generated SQL: SELECT m.Name, SUM(t.AmountINR) AS TotalDebitSpend
FROM Transactions t
JOIN Merchant m ON t.Merchant...
2026-02-11 17:08:11,377 - agent.sql_validator - INFO - Schema validation passed for tables: {'Merchant', 'Transactions'}
2026-02-11 17:08:11,377 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:08:11,378 - agent.db_adapter - INFO - Query executed successfully, returned 5 rows
2026-02-11 17:08:11,379 - agent.executor - INFO - Query executed, returned 5 rows
2026-02-11 17:08:12,881 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:08:12,886 - agent.


EVALUATION SUMMARY
Total Queries:        11
Passed:               10
Failed:               1
Pass Rate:            90.91%
SQL Valid Rate:       100.00%
Avg Response Match:   7.08%


DETAILED EVALUATION RESULTS


Query SQL Valid Response Match  \
0   Top 5 merchants by debit spend in July 2...         ✓            78%   
1        Count active accounts by account type.         ✓             0%   
2   List the top 3 spending categories by to...         ✓             0%   
3   Top 3 customers who spent the most on di...         ✓             0%   
4   Current balance for the top 5 accounts (...         ✓             0%   
5         Give me the total deposits by branch.         ✓             0%   
6   Which branch has the highest number of c...         ✓             0%   
7   Show the top 5 highest-value transaction...         ✓             0%   
8   Top 5 merchants by number of disputes (a...         ✓             0%   
9   Show transaction volume numbers and perc...         ✓             0%   
10  Top 5 Average customer debit spend by me...         ✓             0%   

   Passed Error  
0       ✗  None  
1       ✓  None  
2       ✓  None  
3       ✓  None  
4       ✓  None  
5       ✓  None  
6       ✓  None  
7       ✓  None  
8       ✓  None  
9       ✓  None  
10      ✓  None

## 7. Save Evaluation Results

Save the evaluation results to a CSV file for further analysis.

In [12]:
# Save results
output_path = "evaluation_results.csv"
eval_harness.save_results(results, output_path)
print(f"✓ Results saved to {output_path}")

# Display summary statistics
print(f"\nSummary Statistics:")
print(f"  • Total Queries: {summary['total_queries']}")
print(f"  • Passed: {summary['passed']} ({summary['pass_rate']:.1%})")
print(f"  • Failed: {summary['failed']} ({(1-summary['pass_rate']):.1%})")
print(f"  • SQL Valid: {summary['sql_valid_count']} ({summary['sql_valid_rate']:.1%})")
print(f"  • Avg Response Match: {summary['avg_response_match_score']:.1%}")

2026-02-11 17:09:18,390 - agent.eval - INFO - Results saved to evaluation_results.csv


✓ Results saved to evaluation_results.csv

Summary Statistics:
  • Total Queries: 11
  • Passed: 10 (90.9%)
  • Failed: 1 (9.1%)
  • SQL Valid: 11 (100.0%)
  • Avg Response Match: 7.1%


## 8. Detailed Query Analysis

Analyze specific queries and their generated SQL in detail.

In [13]:
# Function to display detailed query analysis
def analyze_query_detail(result, idx=None):
    print("\n" + "="*70)
    if idx:
        print(f"QUERY ANALYSIS #{idx}")
    print("="*70)
    print(f"\n📝 Original Query:\n   {result['query']}")
    print(f"\n✓ SQL Valid: {result['sql_valid']}")
    
    if result['generated_sql']:
        print(f"\n🗂️  Generated SQL:\n{result['generated_sql']}")
    
    if result['error']:
        print(f"\n❌ Error:\n   {result['error']}")
    else:
        print(f"\n📊 Results: {len(result['results'])} rows returned")
        if result['results']:
            print(f"\n   First few rows:")
            sample_df = pd.DataFrame(result['results'][:5])
            display(sample_df)
    
    if result.get('match_score') is not None:
        print(f"\n📈 Response Match Score: {result['match_score']:.1%}")
    
    print(f"\n✓ Pass: {result['passed']}")

# Analyze first few queries
for i, result in enumerate(results[:3], 1):
    analyze_query_detail(result, i)


QUERY ANALYSIS #1

📝 Original Query:
   Top 5 merchants by debit spend in July 2025.

✓ SQL Valid: True

🗂️  Generated SQL:
SELECT m.Name, SUM(t.AmountINR) AS TotalDebitSpend
FROM Transactions t
JOIN Merchant m ON t.MerchantID = m.MerchantID
WHERE t.TxnType = 'Debit' AND t.TxnDate >= '2025-07-01' AND t.TxnDate < '2025-08-01'
GROUP BY m.Name
ORDER BY TotalDebitSpend DESC
LIMIT 5;

📊 Results: 5 rows returned

   First few rows:


Name  TotalDebitSpend
0         IRCTC        1002190.0
1        Zomato         965256.0
2        Swiggy         965091.0
3   Apple Store         688285.0
4  Amazon India         595250.0


📈 Response Match Score: 77.8%

✓ Pass: False

QUERY ANALYSIS #2

📝 Original Query:
   Count active accounts by account type.

✓ SQL Valid: True

🗂️  Generated SQL:
SELECT AccountType, COUNT(*) AS ActiveAccountCount 
FROM Account 
WHERE Status = 'Active' 
GROUP BY AccountType 
ORDER BY ActiveAccountCount DESC;

📊 Results: 2 rows returned

   First few rows:


AccountType  ActiveAccountCount
0    Checking                  12
1     Savings                  10


📈 Response Match Score: 0.0%

✓ Pass: True

QUERY ANALYSIS #3

📝 Original Query:
   List the top 3 spending categories by total transaction value.

✓ SQL Valid: True

🗂️  Generated SQL:
SELECT Category, SUM(AmountINR) AS TotalSpent
FROM Transactions
GROUP BY Category
ORDER BY TotalSpent DESC
LIMIT 3;

📊 Results: 3 rows returned

   First few rows:


Category  TotalSpent
0       Salary  48387171.0
1       Travel  20653960.0
2  Electronics  18997311.0


📈 Response Match Score: 0.0%

✓ Pass: True


## 9. Test Custom Query

You can write your own natural language query to test the system.

In [14]:
# Test a custom query
custom_query = "Show me customers with highest balance across all accounts"

print(f"\nCustom Query: {custom_query}")
print("-" * 70)

custom_result = executor.execute(custom_query, include_response=True)

analyze_query_detail(custom_result)


Custom Query: Show me customers with highest balance across all accounts
----------------------------------------------------------------------


2026-02-11 17:09:33,732 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:09:33,736 - agent.llm_interface - INFO - Generated SQL: SELECT Customer.CustomerID, Customer.Name, SUM(Transactions.AmountINR) AS TotalBalance
FROM Customer...
2026-02-11 17:09:33,736 - agent.executor - INFO - Generated SQL: SELECT Customer.CustomerID, Customer.Name, SUM(Transactions.AmountINR) AS TotalBalance
FROM Customer...
2026-02-11 17:09:33,737 - agent.sql_validator - INFO - Schema validation passed for tables: {'Account', 'Customer', 'Transactions'}
2026-02-11 17:09:33,737 - agent.sql_validator - INFO - SQL query passed validation
2026-02-11 17:09:33,740 - agent.db_adapter - INFO - Query executed successfully, returned 10 rows
2026-02-11 17:09:33,741 - agent.executor - INFO - Query executed, returned 10 rows
2026-02-11 17:09:37,043 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-11 17:09:37

KeyError: 'query'

## 10. System Architecture Overview

Let's examine the system components and their interactions.

In [ ]:
print("\n" + "="*70)
print("SYSTEM ARCHITECTURE")
print("="*70)

components = {
    "DatabaseAdapter": {
        "role": "Safe read-only SQLite connection",
        "features": ["Readonly mode", "Query timeout", "Row limit", "Schema introspection"]
    },
    "SchemaLoader": {
        "role": "Database schema management",
        "features": ["Table/column extraction", "Schema context generation", "Allowlist management"]
    },
    "LLMInterface": {
        "role": "OpenAI API integration",
        "features": ["SQL generation", "NL response generation", "Environment variable support"]
    },
    "SQLValidator": {
        "role": "Query safety and security",
        "features": ["Forbidden keyword detection", "Table/column validation", "Comment injection prevention"]
    },
    "TextToSQLExecutor": {
        "role": "Main execution pipeline",
        "features": ["Query generation", "Validation", "Safe execution", "Response generation"]
    },
    "EvaluationHarness": {
        "role": "Test and evaluation framework",
        "features": ["Batch evaluation", "Fuzzy matching", "Results CSV export"]
    }
}

for component, info in components.items():
    print(f"\n{component}")
    print(f"  Role: {info['role']}")
    print(f"  Features:")
    for feature in info['features']:
        print(f"    • {feature}")

print("\n" + "="*70)
print("SAFETY FEATURES")
print("="*70)

safety_features = [
    "SELECT-only queries (no INSERT/UPDATE/DELETE/DROP/ALTER)",
    "SQL comment injection prevention",
    "Schema-based table/column allowlist",
    "Read-only SQLite URI mode",
    "Query execution timeout (5 seconds)",
    "Result row limit (1000 rows)",
    "Parameterized query execution",
    "LLM prompt constraints for SQL-only output"
]

for i, feature in enumerate(safety_features, 1):
    print(f"{i}. {feature}")

## 11. Performance and Statistics

Display system performance metrics.

In [15]:
# Calculate performance statistics
print("\n" + "="*70)
print("PERFORMANCE STATISTICS")
print("="*70)

passed_results = [r for r in results if r['passed']]
failed_results = [r for r in results if not r['passed']]
sql_valid_results = [r for r in results if r['sql_valid']]

print(f"\nExecution Statistics:")
print(f"  • Total Queries Evaluated: {len(results)}")
print(f"  • Passed: {len(passed_results)}")
print(f"  • Failed: {len(failed_results)}")
print(f"  • SQL Valid: {len(sql_valid_results)}")

print(f"\nMatch Scores:")
match_scores = [r['match_score'] for r in results if r.get('match_score') is not None]
if match_scores:
    print(f"  • Average: {sum(match_scores)/len(match_scores):.1%}")
    print(f"  • Min: {min(match_scores):.1%}")
    print(f"  • Max: {max(match_scores):.1%}")

print(f"\nError Analysis:")
errors = {}
for r in failed_results:
    if r['error']:
        error_type = r['error'].split(':')[0] if ':' in r['error'] else r['error'][:30]
        errors[error_type] = errors.get(error_type, 0) + 1

if errors:
    for error_type, count in sorted(errors.items(), key=lambda x: -x[1]):
        print(f"  • {error_type}: {count}")
else:
    print(f"  • No errors found")


PERFORMANCE STATISTICS

Execution Statistics:
  • Total Queries Evaluated: 11
  • Passed: 10
  • Failed: 1
  • SQL Valid: 11

Match Scores:
  • Average: 7.1%
  • Min: 0.0%
  • Max: 77.8%

Error Analysis:
  • No errors found


## 12. Cleanup

Close database connection and clean up resources.

In [16]:
# Clean up
db_adapter.disconnect()
print("✓ Database connection closed")
print("✓ Cleanup complete")

print("\n" + "="*70)
print("EVALUATION COMPLETE")
print("="*70)
print(f"\nResults Summary:")
print(f"  • Pass Rate: {summary['pass_rate']:.1%}")
print(f"  • SQL Valid Rate: {summary['sql_valid_rate']:.1%}")
print(f"  • Avg Response Match: {summary['avg_response_match_score']:.1%}")
print(f"\nResults exported to: {output_path}")
print("\nNext Steps:")
print("  1. Review detailed results in evaluation_results.csv")
print("  2. Analyze failed queries for improvement opportunities")
print("  3. Fine-tune SQL generation prompts if needed")
print("  4. Run evaluation_runner script for batch processing")

2026-02-11 17:09:50,024 - agent.db_adapter - INFO - Database connection closed


✓ Database connection closed
✓ Cleanup complete

EVALUATION COMPLETE

Results Summary:
  • Pass Rate: 90.9%
  • SQL Valid Rate: 100.0%
  • Avg Response Match: 7.1%

Results exported to: evaluation_results.csv

Next Steps:
  1. Review detailed results in evaluation_results.csv
  2. Analyze failed queries for improvement opportunities
  3. Fine-tune SQL generation prompts if needed
  4. Run evaluation_runner script for batch processing
